# Constructive Solid Geometry (CSG) (Unit 4.2)

These geometries are bases on primitives (e.g. sphere, cylinder, plane) which are used to build solids by performing boolean operations. Netgen offers the following primitives

<table width="80%" style="border-collapse: collapse">
<tr>
    <th style="border: 1px solid black; text-align: left;">primitive</th>
    <th style="border: 1px solid black; text-align: left;">csg syntax</th>
    <th style="border: 1px solid black; text-align: left;">meaning</th>
</tr>
<tr> 	 	
    <td style="border: 1px solid black; text-align: left;">half-space</td>
    <td style="border: 1px solid black; text-align: left;">Plane(Pnt a,Vec n)</td>
    <td style="border: 1px solid black; text-align: left;">point p in plane, normal vector</td>
</tr>
<tr>
   <td style="border: 1px solid black; text-align: left;">sphere</td>
    <td style="border: 1px solid black; text-align: left;">Sphere(Pnt c,float r)</td>
    <td style="border: 1px solid black; text-align: left;">sphere with center c and radius r</td>
</tr>
<tr>
    <td style="border: 1px solid black; text-align: left;">cylinder</td>
    <td style="border: 1px solid black; text-align: left;">Cylinder(Pnt a, Pnt b, float r)</td>
    <td style="border: 1px solid black; text-align: left;">points a and b define the axes of a infinite cylinder of radius r</td>
</tr>
<tr>
    <td style="border: 1px solid black; text-align: left;">brick </td>
    <td style="border: 1px solid black; text-align: left;">OrthoBrick (Pnt a, Pnt b)</td>
    <td style="border: 1px solid black; text-align: left;">axes parallel brick with minimal coordinates a and maximal coordinates b</td>
</tr>
</table>

and the boolean operators
<table width="40%" style="border-collapse: collapse">
<tr>
    <th style="border: 1px solid black; text-align: center;">operator</th>
    <th style="border: 1px solid black; text-align: center;">set operation</th>
</tr>
<tr>
    <td style="border: 1px solid black; text-align: center;">*</td>
    <td style="border: 1px solid black; text-align: center;">intersection</td>
</tr>
<tr>
    <td style="border: 1px solid black; text-align: center;">+</td>
    <td style="border: 1px solid black; text-align: center;">union</td>
</tr>
<tr>
    <td style="border: 1px solid black; text-align: center;">-</td>
    <td style="border: 1px solid black; text-align: center;">intersection with complement</td>
</tr>
</table>

In [2]:
import netgen.gui
%gui tk
from ngsolve.solve import Draw, Redraw # just for visualization


Using these primitives and operations, we can easily construct a cube. First we import the netgen.csg module, create 6 planes and intersect them to get the solid cube.

In [2]:
from netgen.csg import *

left  = Plane (Pnt(0,0,0), Vec(-1,0,0) )
right = Plane (Pnt(1,1,1), Vec( 1,0,0) )
front = Plane (Pnt(0,0,0), Vec(0,-1,0) )
back  = Plane (Pnt(1,1,1), Vec(0, 1,0) )
bot   = Plane (Pnt(0,0,0), Vec(0,0,-1) )
top   = Plane (Pnt(1,1,1), Vec(0,0, 1) )

cube = left * right * front * back * bot * top

Then we create a CSGeometry object and add the solid.

In [3]:
geo = CSGeometry()
geo.Add (cube)

mesh = geo.GenerateMesh(maxh=0.25)
Redraw()
# mesh.Save("cube.vol")


In [4]:
from netgen.csg import *

cube = OrthoBrick( Pnt(0,0,0), Pnt(1,1,1) )
hole = Cylinder ( Pnt(0.5, 0.5, 0), Pnt(0.5, 0.5, 1), 0.2)

geo = CSGeometry()
geo.Add (cube-hole.maxh(0.05))
mesh = geo.GenerateMesh(maxh=0.1)
Redraw()


## Setting properties of solids

A solid has members which we can set to define the desired properties.


In [5]:
sphere = Sphere(Pnt(0,0,0),1)

Now we can set a boundary name and a maximal mesh size on the surface of this sphere

In [6]:
sphere.bc("sphere").maxh(0.25)

and define a material for the volume

In [7]:
sphere.mat("iron")

In case we want to visualize the geometry we can define the color (using rgb values) and transparency of the solid.

In [8]:
sphere.col([1,0,0])#.transp()

In [9]:
geo = CSGeometry()
geo.Add(sphere)
geo.Draw()
Redraw()



In [10]:
ngmesh = geo.GenerateMesh()
print(type(ngmesh))
Redraw()

<class 'netgen.libngpy._meshing.Mesh'>


To improve the approximation of curved geometries it is possible to use curved elements. This can be done within NGSolve. Thus we have to convert the Netgen mesh to a NGSolve mesh before curving it.

In [11]:
from ngsolve.comp import Mesh
mesh = Mesh(ngmesh)
print(type(mesh))
Redraw()

<class 'ngsolve.comp.Mesh'>


In [12]:
mesh.Curve(3)
Draw(mesh)


## Setting the mesh size

There are the following options to set the mesh size:

- globally as argument maxh of GenerateMesh
- to the surface of one solid (maxh property as above mentioned)
- for the volume of a solid as optional argument when adding it to the geometry Add(...,bc)
- restrict the mesh size for one point using RestrictH
- use CloseSurfaces to generate anisotropic meshes

### Global mesh size

The global mesh size can be set with the named argument maxh. The following two versions are equivalent since all arguments of the of the GenerateMesh function are parsed to the MeshingParameters if no named argument mp is given.


In [13]:
unit_cube.GenerateMesh(maxh=0.4)


In [14]:
mp = MeshingParameters(maxh=0.4)
unit_cube.GenerateMesh(mp = mp)


### Mesh size for one solid

To set the mesh size for one domain of the mesh we have to add the desired maxh as argument when adding the solid to the geometry


In [15]:
geo = CSGeometry()

brick = OrthoBrick(Pnt(-2,-2,-2),Pnt(2,2,2))
sphere = Sphere(Pnt(0,0,0),1)

geo.Add(brick-sphere)
geo.Add(sphere,maxh=0.1)
ngmesh = geo.GenerateMesh(maxh=0.4)


### Mesh size on a surface

If we want to refine just on a surface we define it as property of the solid.


In [16]:
geo = CSGeometry()

brick = OrthoBrick(Pnt(-2,-2,-2),Pnt(2,2,2))
sphere = Sphere(Pnt(0,0,0),1)

geo.Add(brick-sphere)
geo.Add(sphere.maxh(0.1))
ngmesh = geo.GenerateMesh()


### Mesh size in points

This can be done with the MeshingParameters. Using RestrictH we can define the mesh size in an arbitrary point.


In [17]:
geo = CSGeometry()

brick = OrthoBrick(Pnt(-2,-2,-2),Pnt(2,2,2))
sphere = Sphere(Pnt(0,0,0),1)

mp = MeshingParameters(maxh=0.4)
mp.RestrictH (x=0, y=0, z=1, h=0.025)
        
geo.Add(brick-sphere)
geo.Add(sphere)
ngmesh = geo.GenerateMesh(mp = mp)



### Anisotropic meshes

If the geometry contains thin layers we can use CloseSurfaces to avoid elements with small angles.


In [18]:
from netgen.csg import *

geo = CSGeometry()

box = OrthoBrick(Pnt(0,0,0),Pnt(1,1,1))
top = Plane(Pnt(0,0,0.52),Vec(0,0,1))
bot = Plane(Pnt(0,0,0.48),Vec(0,0,-1))
plate = box * top * bot

geo.Add((box-top).mat("air"))
geo.Add(plate.mat("plate"))
geo.Add((box-bot).mat("air"))

slices = [2**(-i) for i in reversed(range(1,6))]
# define the close surfaces
geo.CloseSurfaces(bot,top)#,slices)
nmesh = geo.GenerateMesh(maxh=0.3)
# refine the mesh between the close surfaces
# ZRefinement(nmesh,geo)




## Setting boundary conditions
### Boundary condition on the surface of a solid

Setting a boundary condition on the whole surface of solid can be achieved by adding it as property to the solid.


In [19]:
brick = OrthoBrick(Pnt(-2,-2,-2),Pnt(2,2,2)).bc('outer')
sphere = Sphere(Pnt(0,0,0),1).bc('sphere')


### Modify boundary between two solids

This can be done by adding the named argument bcmod when adding the solid to the geometry. Here we change the boundary condition on the surface between the halfsphere and the already added box.


In [20]:
halfsphere = sphere * Plane(Pnt(0,0,0),Vec(1,0,0)).bc('plane')
box = brick-sphere
geo = CSGeometry()
geo.Add(box.col([1,0,0]).transp())
geo.Add(halfsphere.col([0,0,1]),bcmod=[(box,"halfsphere")])
geo.Draw()

In [21]:
ngmesh = geo.GenerateMesh()
mesh = Mesh(ngmesh)
mesh.GetBoundaries()

('outer',
 'outer',
 'outer',
 'outer',
 'outer',
 'outer',
 'sphere',
 'halfsphere',
 'plane')

## Notes
- It seems that a cylinder in NGSolve has infinite length, even though we specified two points and a radius.  If we just try to mesh it it will fail after using all the memory, but if we intersect it with an orthobrick of the same length, it works.  The points are just to give it an orientation, not a finite length.
- It seems that NGSolve has difficulty with intersecting solids when the region of intersection consists of too few points.  For example, intersecting a cylinder with an orthobox is fine, but intersecting two cylinders of the same radius with perpendicular axes fails.  If we make the radius of one cylinder slightly larger, it works.

## Undocumented Features from Forum
It is possible to rotate a solid like this:

In [1]:
from netgen.meshing import *
from netgen.csg import *

def MakeCylinder(a, b, r):
    cyl = Cylinder(a, b, r)
    top = Plane(b, b-a)
    bot = Plane(a, a-b)
    return cyl*top*bot

geom = CSGeometry()

cyl1 = MakeCylinder(Pnt(0,0,0), Pnt(1,0,0), 0.5)

# define coordinate system with origin and unit vectors ex, ey, ez 
SetTransformation(Pnt(3,0,0), Vec(0,1,0), Vec(0,0,1), Vec(1,0,0))

cyl2 = MakeCylinder(Pnt(0,0,0), Pnt(1,0,0), 0.5)

geom.Add(cyl1+cyl2)
geom.Draw()

print ("Pnt(1,0,0) = ", Pnt(1,0,0))
print ("Vec(0,0,1) = ", Vec(0,0,1))




Pnt(1,0,0) =  (3, 1, 0)
Vec(0,0,1) =  (1, 0, 0)
